<h1>New Experiment 2: Random rejection of motion-free subjects</h1>
In this notebook, random rejection tests were conducted using data from S1 in the still condition. The generalisation tests were conducted over S2. Each time we fix the number of retained volumes N. We randomly select N volumes from a subject's data, then we train our proposed model over this sample. At testing, we applied the same rejection scheme to S2, then test over the subset of S2.</br>
We would like to show this proposed model is robust, by which it is independent of 1. The proportion of discarded volumes and 2. The different combinations of b-values.</br>
<h2>Packages</h2>

In [ ]:
import nibabel as nib
import numpy as np
"""
packages that does conventional model fitting
"""
import amico
"""
packages that generate train/test dataset
"""
from FormatData import generate_data, parser as data_parser
"""
packages that produce the rejection shceme
"""
from filter_qa import parser as filter_parser, load_eddy
"""
package to store the intermediate result
"""
import pickle
"""
package to work out RMSE and SSIM
"""
from utils import calc_RMSE, calc_ssim, load_nii_image

<h2>The class for parsing command line arguments</h2>

In [ ]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

<h2>Data Preprocessing</h2>

In [ ]:
# motion-free subject path
s01_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/'
s02_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/'
# motion-free target labels
s01_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_NDI.nii'
s02_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_NDI.nii'

s01_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_ODI.nii'
s02_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_ODI.nii'

s01_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_FWF.nii'
s02_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_FWF.nii'

In [ ]:
def filter_mask(subpath, fwfpath, threshold=0.99):
    """
    By looking at the imgs generated, we have found out there are some regions that should not be included. Since they have values higher than 1.0
    And we have found out voxels have NDI and ODI values, while that voxel has GROUND TRUTH FWF 1.0
    This should indicate that that voxel should not even be included in the training
    Therefore we want to filter the each subject's mask first, by using their corresponding GROUND TRUTH FWF

    Args:
        subpath (string): the path of the subject folder
        fwfpath (string): the path of the corresponding fwf file
        threshold (float): the thresholds to be used to filter of the mask,
                           a stringnent threshold would be 0.9, the least stringnent threshold is 1.0
                           by default, it is set to 0.99
    """
    # fetch the mask data
    img_mask = nib.load(subpath+'mask-e.nii')
    original_mask = img_mask.get_fdata()
    original_affine = img_mask.affine
    shape = original_mask.shape # retain the shape of the mask
    origin_nonzeros = np.count_nonzero(original_mask)
    print('original mask has: ' + str(origin_nonzeros) + ' of nonzero voxels')
    # fetch the FWF data
    fwf = nib.load(fwfpath).get_fdata()
    # filter
    mask = original_mask.flatten() # this makes a copy of the orginal mask
    fwf = fwf.reshape(mask.shape[0]) # reshape fwf to the corresponding shape
    for i in range(len(mask)):
        # if fwf has high value, means there is no tissue
        # therefore, the voxel should be excluded
        if fwf[i] >= threshold:
            mask[i] = 0.0
    # reshape mask back
    mask = mask.reshape(shape)
    filter_nonzeros = np.count_nonzero(mask)
    print('filtered mask has: ' +str(filter_nonzeros) + ' of nonzero voxels')
    # save the mask
    filter_img = nib.Nifti1Image(mask, original_affine)
    nib.save(filter_img, subpath+'filtered_mask.nii')

In [ ]:
# Use the above code to filter each subject's mask. Store as filtered_mask.nii in each subject folder
filter_mask(s01_path, s01_FWF_path)
filter_mask(s02_path, s02_FWF_path)

In [ ]:
# filtered mask path for each subject
s01_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/filtered_mask.nii'
s02_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/filtered_mask.nii'

In [ ]:
s01_mask = load_nii_image(s01_mask_path)
s02_mask = load_nii_image(s02_mask_path)

In [ ]:
"""
Generate the base dataset for s01_still and s02_still for all NODDI parameters.
"""
cmd = "--base --label_type A --subjects s01_still s02_still"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Using nib to fetch the ground truth img
"""
#load the truth data for subject 1
s01_NDI_img = nib.load(s01_NDI_path)
s01_ODI_img = nib.load(s01_ODI_path)
s01_FWF_img = nib.load(s01_FWF_path)
s01_NDI_affine = s01_NDI_img.affine
s01_ODI_affine = s01_ODI_img.affine
s01_FWF_affine = s01_FWF_img.affine
s01_NDI_img_data = s01_NDI_img.get_fdata()
s01_ODI_img_data = s01_ODI_img.get_fdata()
s01_FWF_img_data = s01_FWF_img.get_fdata()
#load the truth data for subject 2
s02_NDI_img = nib.load(s02_NDI_path)
s02_ODI_img = nib.load(s02_ODI_path)
s02_FWF_img = nib.load(s02_FWF_path)
s02_NDI_affine = s02_NDI_img.affine
s02_ODI_affine = s02_ODI_img.affine
s02_FWF_affine = s02_FWF_img.affine
s02_NDI_img_data = s02_NDI_img.get_fdata()
s02_ODI_img_data = s02_ODI_img.get_fdata()
s02_FWF_img_data = s02_FWF_img.get_fdata()

------------------------
<h2>Random Rejection Scheme Setup</h2>
Let N to be the tested number of retained volumes. For N, 100 subsampled schemes were drawn randomly from the full scheme (with the first b=0 volume and at least two different b values always included). Each subsampled schem was used to evaluate both techniques. For both 3D CNN and AMICO, N are 60, 40 and 30. We would give further undersampled scheme to 3D CNN, N are 20, 16 and 12.

In [ ]:
# a list contains the N for each tested retained volume
# N = [60, 40]
# random_dict = {60:[], 40:[]}
# random_dict

In [ ]:
# Generated the random scheme for each tested volume
# for n in N:
#     # for each retained volume, we create 100 subsamples
#     for i in range(100):
#         ones = np.ones(n) # 1 = retained
#         zeros = np.zeros(96-n)

#         scheme = np.random.choice(np.concatenate([ones,zeros]), 96, replace=False)
#         scheme = ' '.join(map(str, scheme))
#         random_dict[n].append(scheme)

In [ ]:
# save the rejection scheme
# random_60_file = 'random_60.pickle'
# random_40_file = 'random_40.pickle'

# with open(random_60_file, 'wb') as handle:
#     pickle.dump(random_dict[60], handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(random_40_file, 'wb') as handle:
#     pickle.dump(random_dict[40], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# If the random rejection is already set, we can load it
random_60 = []
with (open('/home/vw/Desktop/IndividualProject/MedICSS2021_/Net/random_60.pickle', "rb")) as openfile:
    while True:
        try:
            random_60.append(pickle.load(openfile))
        except EOFError:
            break
random_60 = np.array(random_60[0])

random_40 = []
with (open('/home/vw/Desktop/IndividualProject/MedICSS2021_/Net/random_40.pickle', "rb")) as openfile:
    while True:
        try:
            random_40.append(pickle.load(openfile))
        except EOFError:
            break
random_40 = np.array(random_40[0])

In [ ]:
amico_rmse_dict = {60:[], 40:[]}
amico_ssim_dict = {60:[], 40:[]}

----------------------------
<h2>Model fit AMICO</h2>

In [ ]:
def writefile(path, file, combine, savename):
    with open(path+savename, 'w') as fout:
        read_path = path+file
        read_file = open(read_path, 'r')
        lines = read_file.readlines()
        for line in lines:
            temp = line.split()
            temp = [e for e, b in zip(temp, combine) if b == 1]
            fout.write(' '.join(e for e in temp))
            fout.write('\n')
    fout.close()

def writediffusion(path, file, combine):
    img = nib.load(path+file)
    data = img.get_fdata()
    data = data[..., combine==1]
    img = nib.Nifti1Image(data, np.eye(4))
    nib.save(img, path+'remained_diffusion')

----------------------
<h2>N=60</h2>

In [ ]:
for i in range(len(random_60)//2):

    # read the movefile
    movefile = random_60[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])
    
    # create the remained bvals and bvecs for s02 by applying its corresponding rejection scheme
    s02_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/'
    # write the remained bvals
    writefile(s02_path, 'bvals', combine, savename='remained_bvals')
    # write the remained bvecs
    writefile(s02_path, 'bvecs', combine, savename='remained_bvecs')
    # create the remained diffusion data
    writediffusion(s02_path, 'diffusion.nii', combine)

    amico.setup()
    # generate a scheme file from the bvals/bvecs files as follows, using the remained bvals and bvecs. Because data rejection is applied
    amico.util.fsl2scheme(s02_path+'remained_bvals', s02_path+'remained_bvecs')
    ae = amico.Evaluation("/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI", "s02_still")
    # load the data
    ae.load_data(dwi_filename = "remained_diffusion.nii", scheme_filename = "remained_bvals.scheme", mask_filename = "filtered_mask.nii", b0_thr = 0)
    # Set model for NODDI and generate the response functions for all the compartments:
    ae.set_model("NODDI")
    ae.generate_kernels()
    ae.load_kernels()
    # model fit. It takes a little time depending on the number of voxels (but much much faster than the original NODDI).
    ae.fit()
    # Finally, save the results as NIfTI images:
    # ICVF = NDI
    # ISOVF = FWF
    # OD = ODI
    ae.save_results()

    # analyse the result
    icvf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_ICVF.nii.gz'
    isovf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_ISOVF.nii.gz'
    od_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_OD.nii.gz'

    icvf_img = nib.load(icvf_path)
    icvf = icvf_img.get_fdata()
    isovf_img = nib.load(isovf_path)
    isovf = isovf_img.get_fdata()
    od_img = nib.load(od_path)
    od = od_img.get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s02_NDI_img_data, s02_mask)
    odi_rmse = calc_RMSE(od, s02_ODI_img_data, s02_mask)
    fwf_rmse = calc_RMSE(isovf, s02_FWF_img_data, s02_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s02_NDI_img_data)
    odi_ssim = calc_ssim(od, s02_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s02_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[60].append(RMSE)
    amico_ssim_dict[60].append(SSIM)

In [ ]:
with open('ami_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('ami_ssim_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_ssim_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_ssim_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_60)//2, len(random_60)):

    # read the movefile
    movefile = random_60[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])
    
    # create the remained bvals and bvecs for s02 by applying its corresponding rejection scheme
    s02_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/'
    # write the remained bvals
    writefile(s02_path, 'bvals', combine, savename='remained_bvals')
    # write the remained bvecs
    writefile(s02_path, 'bvecs', combine, savename='remained_bvecs')
    # create the remained diffusion data
    writediffusion(s02_path, 'diffusion.nii', combine)

    amico.setup()
    # generate a scheme file from the bvals/bvecs files as follows, using the remained bvals and bvecs. Because data rejection is applied
    amico.util.fsl2scheme(s02_path+'remained_bvals', s02_path+'remained_bvecs')
    ae = amico.Evaluation("/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI", "s02_still")
    # load the data
    ae.load_data(dwi_filename = "remained_diffusion.nii", scheme_filename = "remained_bvals.scheme", mask_filename = "filtered_mask.nii", b0_thr = 0)
    # Set model for NODDI and generate the response functions for all the compartments:
    ae.set_model("NODDI")
    ae.generate_kernels()
    ae.load_kernels()
    # model fit. It takes a little time depending on the number of voxels (but much much faster than the original NODDI).
    ae.fit()
    # Finally, save the results as NIfTI images:
    # ICVF = NDI
    # ISOVF = FWF
    # OD = ODI
    ae.save_results()

    # analyse the result
    icvf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_ICVF.nii.gz'
    isovf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_ISOVF.nii.gz'
    od_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_OD.nii.gz'

    icvf_img = nib.load(icvf_path)
    icvf = icvf_img.get_fdata()
    isovf_img = nib.load(isovf_path)
    isovf = isovf_img.get_fdata()
    od_img = nib.load(od_path)
    od = od_img.get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s02_NDI_img_data, s02_mask)
    odi_rmse = calc_RMSE(od, s02_ODI_img_data, s02_mask)
    fwf_rmse = calc_RMSE(isovf, s02_FWF_img_data, s02_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s02_NDI_img_data)
    odi_ssim = calc_ssim(od, s02_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s02_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[60].append(RMSE)
    amico_ssim_dict[60].append(SSIM)

In [ ]:
with open('ami_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('ami_ssim_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_ssim_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_ssim_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

------
<h2>N=40</h2>

In [ ]:
for i in range(len(random_40)//2):

    # read the movefile
    movefile = random_40[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])

    # for j in range(1, len(combine)):
    #     if combine[j] == 1:
    #         combine[j] = 0
    
    # # the first volume, b=0 is always selected
    # combine[0] = 1
    
    # create the remained bvals and bvecs for s02 by applying its corresponding rejection scheme
    s02_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/'
    # write the remained bvals
    writefile(s02_path, 'bvals', combine, savename='remained_bvals')
    # write the remained bvecs
    writefile(s02_path, 'bvecs', combine, savename='remained_bvecs')
    # create the remained diffusion data
    writediffusion(s02_path, 'diffusion.nii', combine)

    amico.setup()
    # generate a scheme file from the bvals/bvecs files as follows, using the remained bvals and bvecs. Because data rejection is applied
    amico.util.fsl2scheme(s02_path+'remained_bvals', s02_path+'remained_bvecs')
    ae = amico.Evaluation("/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI", "s02_still")
    # load the data
    ae.load_data(dwi_filename = "remained_diffusion.nii", scheme_filename = "remained_bvals.scheme", mask_filename = "filtered_mask.nii", b0_thr = 0)
    # Set model for NODDI and generate the response functions for all the compartments:
    ae.set_model("NODDI")
    ae.generate_kernels()
    ae.load_kernels()
    # model fit. It takes a little time depending on the number of voxels (but much much faster than the original NODDI).
    ae.fit()
    # Finally, save the results as NIfTI images:
    # ICVF = NDI
    # ISOVF = FWF
    # OD = ODI
    ae.save_results()

    # analyse the result
    icvf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_ICVF.nii.gz'
    isovf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_ISOVF.nii.gz'
    od_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_OD.nii.gz'

    icvf_img = nib.load(icvf_path)
    icvf = icvf_img.get_fdata()
    isovf_img = nib.load(isovf_path)
    isovf = isovf_img.get_fdata()
    od_img = nib.load(od_path)
    od = od_img.get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s02_NDI_img_data, s02_mask)
    odi_rmse = calc_RMSE(od, s02_ODI_img_data, s02_mask)
    fwf_rmse = calc_RMSE(isovf, s02_FWF_img_data, s02_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s02_NDI_img_data)
    odi_ssim = calc_ssim(od, s02_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s02_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[60].append(RMSE)
    amico_ssim_dict[60].append(SSIM)

In [ ]:
with open('ami_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('ami_ssim_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_ssim_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_ssim_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_40)//2, len(random_40)):

    # read the movefile
    movefile = random_40[i]
    movefile = movefile.split()
    combine = np.array([int(float(num)) for num in movefile])
    
    # create the remained bvals and bvecs for s02 by applying its corresponding rejection scheme
    s02_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/'
    # write the remained bvals
    writefile(s02_path, 'bvals', combine, savename='remained_bvals')
    # write the remained bvecs
    writefile(s02_path, 'bvecs', combine, savename='remained_bvecs')
    # create the remained diffusion data
    writediffusion(s02_path, 'diffusion.nii', combine)

    amico.setup()
    # generate a scheme file from the bvals/bvecs files as follows, using the remained bvals and bvecs. Because data rejection is applied
    amico.util.fsl2scheme(s02_path+'remained_bvals', s02_path+'remained_bvecs')
    ae = amico.Evaluation("/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI", "s02_still")
    # load the data
    ae.load_data(dwi_filename = "remained_diffusion.nii", scheme_filename = "remained_bvals.scheme", mask_filename = "filtered_mask.nii", b0_thr = 0)
    # Set model for NODDI and generate the response functions for all the compartments:
    ae.set_model("NODDI")
    ae.generate_kernels()
    ae.load_kernels()
    # model fit. It takes a little time depending on the number of voxels (but much much faster than the original NODDI).
    ae.fit()
    # Finally, save the results as NIfTI images:
    # ICVF = NDI
    # ISOVF = FWF
    # OD = ODI
    ae.save_results()

    # analyse the result
    icvf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_ICVF.nii.gz'
    isovf_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_ISOVF.nii.gz'
    od_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/AMICO/NODDI/FIT_OD.nii.gz'

    icvf_img = nib.load(icvf_path)
    icvf = icvf_img.get_fdata()
    isovf_img = nib.load(isovf_path)
    isovf = isovf_img.get_fdata()
    od_img = nib.load(od_path)
    od = od_img.get_fdata()

    # work out RMSE and SSIM
    RMSE = []
    ndi_rmse = calc_RMSE(icvf, s02_NDI_img_data, s02_mask)
    odi_rmse = calc_RMSE(od, s02_ODI_img_data, s02_mask)
    fwf_rmse = calc_RMSE(isovf, s02_FWF_img_data, s02_mask)
    RMSE.append(ndi_rmse)
    RMSE.append(odi_rmse)
    RMSE.append(fwf_rmse)

    SSIM = []
    ndi_ssim = calc_ssim(icvf, s02_NDI_img_data)
    odi_ssim = calc_ssim(od, s02_ODI_img_data)
    fwf_ssim = calc_ssim(isovf, s02_FWF_img_data)
    SSIM.append(ndi_ssim)
    SSIM.append(odi_ssim)
    SSIM.append(fwf_ssim)

    amico_rmse_dict[60].append(RMSE)
    amico_ssim_dict[60].append(SSIM)

In [ ]:
with open('ami_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('ami_ssim_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in amico_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_ssim_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in amico_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('ami_ssim_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in amico_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)